# Multi Mode Source Extraction
----
Pushing the limits of SExtractor




_Massimo Bortone, ETH Zürich_

# Outline
----
1. Goals
2. SExtractor
3. Overshredding
4. Kernels
5. Multi Mode Source Extraction
6. What's next

# Goals
----
* Detect as many possible sources in deep images like the XDF
* Overshred: detect sub-structure in galaxies
* Combine detections from multiple bands and modes
* Forward model the image with FORCEPho

# The HXDF
----
<div style="width: 100%;">
    <div style="float: left; width: 50%">
        <img src="images/xdf.png" alt="XDF and cutout region" />
    </div>
    <ul style="float: left; width: 45%">
        <li>$20 ~\text{days}$ total exposure</li>
        <li>$2.3\times 2 ~\text{arcminutes}^2$ in size, which is about $\frac{1}{32\times 10^6}$ of the sky</li>
        <li>$\text{RA}~~ 03:32:40.440$ <br> $\text{DEC}~~ -27:47:24.12$</li>
        <li>generated using data from Hubble's ACS/WFC and WFC3/IR cameras</li>
    </ul>
</div>

![cutout bands](images/H-I-bands.png "H and I bands of the XDF")

# SExtractor
----
Developed in 1996 by E. Bertin and S. Arnouts, is a parametric signal-based method to extract sources from large astronomical images.

## How it works
----
The complete analysis of an image consists of 6 steps:
1. estimation of sky background
2. **thresholding**
3. **deblending**
4. cleaning
5. photometry
6. star/galaxy separation

1. assume image is already background subtracted or background is sufficiently flat
4. not relevant for our goals
5. handled by FORCEPho
6. not important in deep images

## Thresholding
----
* convolve original image "on-the-fly" with a mask 
* thresholding is local for every pixel:
$$I_{ij} > \sigma \cdot \text{rms}_{ij}$$
* extract 8-connected contigous pixels above threshold from convolved image

## Deblending
----
<div style="width: 100%;">
    <ul style="float: left; width: 60%">
        <li>re-threshold each extracted set of 8-connected pixels at `NTHRESH` exponentially spaced levels between the primary extraction threshold $\sigma$ and the peak value in the extracted set</li>
        <li>model light distribution of the object as a tree structure</li>
        <li>`MINCONT` parameter defines the minimal contrast between the integrated pixel intensity in the branches and the total intensity of the composite object</li>
    </ul>
    <div style="float: left; width: 35%">
        <img src="images/sextractor-deblending.png" alt="Deblending in SExtractor" />
    </div>
</div>

# Overshredding
----
Apart from $\sigma$, the detection behavior of SExtractor is controlled by
- `MINAREA`: minimal number of pixels that define an object
- `NTHRESH`: number of exponentially spaced levels
- `MINCONT`: minimal contrast needed to split an object in two components

1. is quite straightforward: the smaller, the more "objects" are detected
2. & 3. not clear which values are good for our goals

## Exploring the parameter space
----
Since it is not clear which combination of values of `NTHRESH` and `MINCONT` yield to overshredding, let's explore the parameter space

In [ ]:
nthresh = range(32, 1025, 32)
mincont = np.logspace(0, -10, 100)

![grid](images/mincont-nthresh-grid.png "Detections as a function of NTHRESH and MINCONT")

**1. CONCLUSION** 

`MINCONT` has a larger effect on the number of detections than `NTHRESH`

![grid](images/detection-vs-mincont.png "Detections as a function MINCONT for NTHRESH=64")

**2. CONCLUSION** 

Number of detections saturates for `MINCONT`$<10^{-6}$

--------
From now on we fix the parameters to:
- `NTHRESH`=$64$
- `MINCONT`=$10^{-7}$

# Kernel
----
A kernel is a matrix of a given size that is convolved with the image to increase the sensibility of the detection towards specific objects

## Examples

* PSF kernel: good for faint unresolved objects
* flat kernel: better for LSB objects

In [ ]:
psf_filename = os.path.join(base, "data/ZC404221_H_PSF.fits")
psf_hdu = fits.open(psf_filename)
psf = psf_hdu[0].data
psf_clipped = np.array(psf[40:60, 40:60])

In [ ]:
psf_default = np.zeros_like(psf_clipped)
psf_default[9:12, 9] = [1, 2, 1]
psf_default[9:12, 10] = [2, 4, 2]
psf_default[9:12, 11] = [1, 2, 1]
psf_default /= 4.0

In [ ]:
flat_5 = np.zeros_like(psf_clipped)
flat_5[8:13, 8:13] = np.ones((5, 5))

In [ ]:
one_hot = np.zeros_like(psf_clipped)
one_hot[10, 10] = 1

![Kernels](images/kernels.png "4 different kernels")

Comparing the number of detections as a function of area for the 4 different kernels we notice:

![Histograms](images/hist-kernels.png "Histograms of detections for 4 different kernels")

- `ONE-HOT` kernel detects a lot of small scale structure
- `PSF` and `FLAT` kernels are sensible to more extended objects

Overplotting all detections we conclude that:
![All detections](images/all-kernels.png "Detected sources for 4 different kernels")

- SExtractor with the `ONE-HOT` kernel can be use to detect substructures in bright extended objects

# Multi Mode Source Extractor
----
**IDEA:** 
run SExtractor in different modes and on different bands, allowing each mode to specify its own SExtractor parameters. Then merge detections into one single catalog.

----
**PROBLEM:** 
merging detections can be tricky

## Defining modes and bands
----

In [3]:
modes = {
    'H': {
        # detect large bright sources
        'cold': {
            'sigma': 1.5,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_cold,
            'minarea': 5
        },
        # detect small isolated sources
        'hot': {
            'sigma': 1.5,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_hot,
            'minarea': 5
        }
    },
    'I': {
        # detect substructure in large bright sources
        'hot': {
            'sigma': 2.0,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_hot,
            'minarea': 5
        }
    }
}

NameError: name 'kernel_cold' is not defined

In [ ]:
bands = {
    'H': {
        'image': image_h,
        'rms': rms_h
    },
    'I': {
        'image': image_i,
        'rms': rms_i
    }
}

## Finding substructures
----
The idea is to use a small kernel on the I band to detect sub-structures within the brightest objects detected in the H band. To do this we need to:

1. Compute fluxes for all object
2. Select brightest objects in the H band
3. Look for sub-structures in the I band

![H and I detections](images/H-I-detections.png "Detections in the H and I bands")

### 1. Compute flux within elliptical aperture for each detection
![Integrated flux](images/H-integrated-flux.png "Integrated flux for detections in the H band")

### 2. Select top 20% brightest objects in the H band
![Top 20% detections](images/H-flux-selection.png "Top 20% brightest detections in the H band")

### 3. Search for detections in the I band that are within an elliptical aperture of 2x the dimensions determined by SExtractor
![I Sub-structures](images/I-band-substructure.png "Sub-structure detection in the I band")

### 4. Repeat same steps to find sub-structures on the H band
![H Sub-structures](images/H-band-substructure.png "Sub-structure detection in the H band")

### 5. Compare sub-structures in both bands
![Compare sub-structures](images/H-vs-I-substructure.png "Comparison between substructure detected in the H and I bands")

## Merging
----
In order to merge the detections obtained for the different bands and modes we need to:

1. Remove duplicates across bands and modes
2. Remove brightest objects in H band
3. Replace sub-structures in H band with those found in the I band

### Finding similar detections
----
Given two objects $A$ and $B$, we define a proximity function as follows:

$$
   P(A, B) = \parallel x_A - x_B \parallel + \vert s_A - s_B \vert + \vert \theta_A - \theta_B \vert
$$

where $x_i$ are the coordinates of the barycentre (first moment), $s_i = a_i \cdot b_i$ is the area of the ellipse and $\theta_i$ its orientation.

### 1.1 Remove duplicates across modes in the H band

![Duplicates H band](images/H-band-hot-cold-similar.png "Duplicates across modes in the H band")

### 1.2 Remove duplicates across H and I bands in hot mode

![Duplicates hot mode](images/H-I-band-hot-hot-similar.png "Duplicates across H and I bands in hot mode")

## Final positions
----
![Final posititons](images/final-positions.png "Final positions")

## Comparison to other catalogs
----
We compare our detections to those found by:
- Bouwens+ 2015 (see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJ/803/34)
- Guo+ 2013 (CANDELS GOODS-S catalog, see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJS/207/24)
- Finkelstein+ 2015 (see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJ/810/71)
- Skelton+2014 (3D-HST catalog, see http://adsabs.harvard.edu/abs/2014ApJS..214...24S)

Overplotting all detections we find:
    
![All catalogs](images/all-mmse.png "Detections from all catalogs")

**Results**:

![Detection fraction](images/detection-fraction.png "Detection fraction")

**NOTE**: agreement with `Bouwens` can be improved by stacking YJH bands together

# What's next
----

- measure distance between peak and barycentre in each detected object to determine the width of a prior on its final position
- determine detection recovery as a function of confusion by introducing artificial objects on the image
- using simulated images:
    - quantify false positive and true negative detections
    - find optimal kernels for different kinds of objects